## Imports

In [ ]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

In [ ]:
import numpy as np                                                              
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf                                                         
import data.data_selector as ds                                                   
import analysis.analysis_picker as ap
import utils.plot_functions as pf                                               

## Model parameters

In [ ]:
class analysis_params(object):
  def __init__(self):
    self.model_type = "lca_subspace"
    self.model_name = "lca_subspace_mnist"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

# Computed params
params = analysis_params()
params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+params.model_name)

## Load analyzer

In [ ]:
analyzer = ap.get_analyzer(params.model_type)
analyzer.setup(params)
analyzer.setup_model(analyzer.model_params)
analyzer.load_analysis(save_info=params.save_info)
analyzer.model_name = params.model_name

### New Analysis

### MNIST

### VanHateren

## Construct analysis plots

In [ ]:
fig = pf.plot_loc_freq_summary(analyzer.bf_stats)
fig.savefig(analyzer.analysis_out_dir+"vis/location_frequency_centers.png")

In [ ]:
neuron_angles = analyzer.neuron_angles(analyzer.bf_stats) * (180/np.pi)

In [ ]:
angle_hist_fig = pf.plot_weight_angle_histogram(neuron_angles, num_bins=50, angle_min=0, angle_max=180, figsize=(8,8))
angle_hist_fig.savefig(analyzer.analysis_out_dir+"/vis/neuron_angle_histogram.png")

In [ ]:
weight_indices = np.stack([np.array(id_list) for id_list in analyzer.model.module.group_ids], axis=0)
pooling_weights = np.stack([analyzer.evals["lca_subspace/weights/w:0"][:, id_list] for id_list in analyzer.model.module.group_ids], axis=1)